In [10]:
import pandas as pd

# Compensation data
import pandas as pd

df_comp = pd.DataFrame({
    'Staff First Name': ['John', 'Alice', 'Bob'],
    'Staff Last Name': ['Doe', 'Smith', 'Lee'],
    'Location ID': ['LOC1', 'LOC1', 'LOC2'],
    'Schedule Task Name': ['Task A', 'Task B', 'Task C'],
    'Schedule Start Date': pd.to_datetime(['2024-06-03', '2024-06-03', '2024-06-04']),
    'Schedule Start Time': ['08:00', '09:00', '10:00'],
    'Schedule End Date': pd.to_datetime(['2024-06-03', '2024-06-03', '2024-06-04']),
    'Schedule End Time': ['16:00', '13:00', '18:00'],
    'Schedule Duration (hours)': [8.0, 4.0, 8.0]
})


# Date conversions
df_comp['Schedule Start Date'] = pd.to_datetime(df_comp['Schedule Start Date'])
df_comp['Schedule End Date'] = pd.to_datetime(df_comp['Schedule End Date'])
display(df_comp)


,Staff First Name,Staff Last Name,Location ID,Schedule Task Name,Schedule Start Date,Schedule Start Time,Schedule End Date,Schedule End Time,Schedule Duration (hours)
0,John,Doe,LOC1,Task A,2024-06-03,08:00,2024-06-03,16:00,8.0
1,Alice,Smith,LOC1,Task B,2024-06-03,09:00,2024-06-03,13:00,4.0
2,Bob,Lee,LOC2,Task C,2024-06-04,10:00,2024-06-04,18:00,8.0


In [11]:
# MCR data
df_mcr = pd.DataFrame({
    'DOS': pd.to_datetime([
        '2024-06-03', '2024-06-03', '2024-06-03',  # John Doe (2 cases)
        '2024-06-03',  # Alice Smith
        '2024-06-04'   # Bob Lee
    ]),
    'FACILITY': ['LOC1', 'LOC1', 'LOC1', 'LOC1', 'LOC2'],
    'PROVIDER': ['John Doe', 'John Doe', 'Alice Smith', 'John Doe', 'Bob Lee'],
    'PROVIDER 2': [None, 'Alice Smith', None, None, None],
    'PROVIDER TYPE': ['Surgeon'] * 5,
    'Surgeon': ['Yes'] * 5,
    'PATIENT': ['P1', 'P2', 'P3', 'P4', 'P5'],
    'START TIME': ['08:00', '10:00', '09:00', '14:00', '10:30'],
    'END TIME': ['09:00', '11:00', '10:00', '16:00', '12:30'],
    'CALCULATED TIME': [1.0, 1.0, 1.0, 2.0, 2.0],
    'FILE NAME': ['F1.pdf']*5,
    '# OF PAGES': [2]*5,
    'NOTES': ['']*5
})

df_mcr['DOS'] = pd.to_datetime(df_mcr['DOS'])
display(df_mcr)


,DOS,FACILITY,PROVIDER,PROVIDER 2,PROVIDER TYPE,Surgeon,PATIENT,START TIME,END TIME,CALCULATED TIME,FILE NAME,# OF PAGES,NOTES
0,2024-06-03,LOC1,John Doe,None,Surgeon,Yes,P1,08:00,09:00,1.0,F1.pdf,2,
1,2024-06-03,LOC1,John Doe,Alice Smith,Surgeon,Yes,P2,10:00,11:00,1.0,F1.pdf,2,
2,2024-06-03,LOC1,Alice Smith,None,Surgeon,Yes,P3,09:00,10:00,1.0,F1.pdf,2,
3,2024-06-03,LOC1,John Doe,None,Surgeon,Yes,P4,14:00,16:00,2.0,F1.pdf,2,
4,2024-06-04,LOC2,Bob Lee,None,Surgeon,Yes,P5,10:30,12:30,2.0,F1.pdf,2,


In [12]:
import pandas as pd

# Step 1: Unpivot MCR data and clean up provider names
df_mcr_long = pd.melt(
    df_mcr,
    id_vars=['DOS', 'FACILITY', 'PATIENT', 'START TIME', 'END TIME', 'CALCULATED TIME', 'FILE NAME', '# OF PAGES', 'NOTES', 'PROVIDER TYPE', 'Surgeon'],
    value_vars=['PROVIDER', 'PROVIDER 2'],
    var_name='Provider Role',
    value_name='PROVIDER_NAME'
)

# Filter out rows where provider is missing or invalid
df_mcr_long = df_mcr_long[df_mcr_long['PROVIDER_NAME'].notna()]
df_mcr_long = df_mcr_long[df_mcr_long['PROVIDER_NAME'].str.split().str.len() == 2]

# Extract First and Last Name
df_mcr_long[['Staff First Name', 'Staff Last Name']] = df_mcr_long['PROVIDER_NAME'].str.split(' ', expand=True)

# Add month for join key
df_mcr_long['Month'] = pd.to_datetime(df_mcr_long['DOS']).dt.to_period('M').astype(str)
df_mcr_long['FACILITY'] = df_mcr_long['FACILITY'].astype(str)

# Add month to compensation data
df_comp['Month'] = pd.to_datetime(df_comp['Schedule Start Date']).dt.to_period('M').astype(str)
df_comp['Location ID'] = df_comp['Location ID'].astype(str)

# Step 2: Join on Name, Location, and Month
df_result = pd.merge(
    df_mcr_long,
    df_comp,
    how='left',
    left_on=['Staff First Name', 'Staff Last Name', 'FACILITY', 'Month'],
    right_on=['Staff First Name', 'Staff Last Name', 'Location ID', 'Month']
)

# Step 3: Clean and format final output
df_result['Actual Worked Hours'] = df_result['CALCULATED TIME']

final_columns = [
    'Staff First Name', 'Staff Last Name', 'Location ID', 'Month',
    'Schedule Task Name', 'Schedule Start Date', 'Schedule Start Time',
    'Schedule End Date', 'Schedule End Time', 'Schedule Duration (hours)',
    'DOS', 'Actual Worked Hours'
]

df_result_final = df_result[final_columns].sort_values(
    by=['Staff First Name', 'Month', 'Location ID', 'DOS']
).reset_index(drop=True)


In [13]:
display(df_result_final)

,Staff First Name,Staff Last Name,Location ID,Month,Schedule Task Name,Schedule Start Date,Schedule Start Time,Schedule End Date,Schedule End Time,Schedule Duration (hours),DOS,Actual Worked Hours
0,Alice,Smith,LOC1,2024-06,Task B,2024-06-03,09:00,2024-06-03,13:00,4.0,2024-06-03,1.0
1,Alice,Smith,LOC1,2024-06,Task B,2024-06-03,09:00,2024-06-03,13:00,4.0,2024-06-03,1.0
2,Bob,Lee,LOC2,2024-06,Task C,2024-06-04,10:00,2024-06-04,18:00,8.0,2024-06-04,2.0
3,John,Doe,LOC1,2024-06,Task A,2024-06-03,08:00,2024-06-03,16:00,8.0,2024-06-03,1.0
4,John,Doe,LOC1,2024-06,Task A,2024-06-03,08:00,2024-06-03,16:00,8.0,2024-06-03,1.0
5,John,Doe,LOC1,2024-06,Task A,2024-06-03,08:00,2024-06-03,16:00,8.0,2024-06-03,2.0
